In [21]:
using Distributed

using Pkg

Pkg.instantiate()

@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

   Resolving package versions...


  No Changes to `~/ParallelTemperingMonteCarlo.jl/Project.toml`
  No Changes to `~/ParallelTemperingMonteCarlo.jl/Manifest.toml`


Precompiling project...


  ✓ ParallelTemperingMonteCarlo
  1 dependency successfully precompiled in 3 seconds. 164 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version


In [8]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [18]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

# runnerdir = "/home/ghun245/RuNNer-master/Brass_potential/"
# atomtype="Cu"
# pot = AbstractMLPotential(runnerdir,atomtype)

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
# mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

# #results = Output(n_bin, max_displ_vec)
# results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

# cd("/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts")



100

In [13]:
# function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
#     if isdir("pots") == true
#         rm("pots",recursive = true)
#     end
#     potential_directory = mkdir("$(pwd())/pots")
#     potential_vector = []
#     for i = 1:n_threads
#         # n_threads potentials
#         cp(pot.dir,"$(potential_directory)/potential$i")
#         pot_i = AbstractMLPotential("$(potential_directory)/potential$i/",pot.atomtype)
#         push!(potential_vector,pot_i)
#     end
        
#     return potential_vector        
# end

# function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
#     parallel_states = []

#     pot_vector = init_parallel_RuNNer(pot) #initialise state and potentials

#     n_threads = Threads.nthreads()
#     sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

#     a = atom_move_frequency(move_strat)
#     v = vol_move_frequency(move_strat)
#     r = rot_move_frequency(move_strat)
#     #number of MC steps per MC cycle
#     n_steps = a + v + r

#     println("Total number of moves per MC cycle: ", n_steps)
#     println()

#     ebounds = [100. , -100.]

#     for i_thread = 1:n_threads
#         for i_eq = 1:sample_index
            
#             i = i_thread*sample_index + i_eq

#             mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot_vector[i_thread], ensemble, n_steps, a, v, r)#mc cycle
            
#             for i_traj = 1:mc_params.n_traj#check energy bounds
#                 if mc_states[i_traj].en_tot < ebounds[1]
#                     ebounds[1] = mc_states[i_traj].en_tot
#                 end

#                 if mc_states[i_traj].en_tot > ebounds[2]
#                     ebounds[2] = mc_states[i_traj].en_tot
#                 end

#             end

#             if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
#                 for i_traj = 1:mc_params.n_traj
#                     update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
#                 end 
#             end

#         end
        
#         states_vec = [MCState(mc_states[i_traj].temp,mc_states[i_traj].beta,mc_states[i_traj].config,pot_vector[i_thread];max_displ = mc_states[i_traj].max_displ ) for i_traj in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

#         for i_traj = 1:mc_params.n_traj
#             push!(states_vec[i_traj].ham, 0)
#             push!(states_vec[i_traj].ham, 0)
#         end

#         push!(parallel_states,states_vec) #add to vector of parallel states


#     end
    
    

#     delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

#     println("equilibration done")
    
#     return parallel_states,pot_vector,a,v,r,delta_en,n_threads

# end

parallel_equilibration (generic function with 1 method)

In [14]:
@everywhere function threadexchange(parallel_states,n_threads,idx)
    if rand() < 0.2  #20% change per trajectory of an attempted exchange
        thrid = rand(1:n_threads,2)
        if thrid[1] == thrid[2] && thrid[2] == n_threads
            thrid[2] = rand(1:n_threads-1)
        elseif thrid[1] == thrid[2] && thrid[2] != n_threads
            thrid[2] +=1
        end #which threads are talking

        exc_acc = exc_acceptance(parallel_states[thrid[1]][idx].beta,parallel_states[thrid[2]][idx].beta,parallel_states[thrid[1]][idx].en_tot,parallel_states[thrid[2]][idx].en_tot) #calc acceptance
        
        if exc_acc > rand() #if exchange is likely
            exc_trajectories!(parallel_states[thrid[1]][idx] ,parallel_states[thrid[2]][idx] )#swap
        end
     end
    
end



In [9]:
runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
atomtype="Cu"
pot = AbstractMLPotential(runnerdir,atomtype)
RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)

AbstractMLPotential("/home/ghun245/RuNNer-testing/Brass_potential/", "Cu")

In [11]:
struct ParallelMLPotential <: AbstractPotential
    dir::String
    atomtype::String
    index::Int64
end
function MCState(temp,beta, config::Config, pot::ParallelMLPotential;kwargs...)
    dist2_mat = get_distance2_mat(config)
    n_atoms = length(config.pos)
    en_atom_vec = zeros(n_atoms)
    
    en_tot = RuNNer.getenergy(pot.dir, config,pot.atomtype,pot.index)


    MCState(temp, beta, config, dist2_mat, en_atom_vec, en_tot; kwargs...)
end

MCState

In [16]:
pot_dir = []
for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_dir,potential)
end


In [22]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_dir[i]; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

results = Output(n_bin, max_displ_vec)

MethodError: MethodError: no method matching getenergy(::String, ::Config{13, SphericalBC{Float64}, Float64}, ::String, ::Int64)
Closest candidates are:
  getenergy(::Any, ::Config, ::Any, ::Any, !Matched::StaticArraysCore.SVector) at ~/ParallelTemperingMonteCarlo.jl/src/RuNNer.jl:235
  getenergy(::Any, ::Config, ::Any) at ~/ParallelTemperingMonteCarlo.jl/src/RuNNer.jl:227